## Ch 4. Variational Autoencoders

- VAE를 위한 강의였고, VAE의 설명과 논문들이 저자마다 너무 수학을 잘하는 사람들이어서 의미를 이해하기 힘들다. 
- AE는 압축할 때 가장 많이 사용하고 AE 태워보고, 다시 한번더 고전적인 t-sne와 같은 방법으로 시각화하여 사용할지 말지 결정한다.


- VAE는 AE와 코드를 보면 거의 똑같지만, 논문의 흐름을 보면 AE와 전혀 관계가 없는 내용이다.
    - AE는 목적인 Encoder를 학습하기위해(Manifold를 잘 학습하기 위해) Decoder를 붙였던 것이고, VAE는 Generation이 목적으로 반대로 Generation을 잘 학습하기 위해 Encoder가 붙여 전체를 보니까 AE랑 구조가 같아진 것이다.


- 우리가 generate를 할 때 latent에서 control 하고싶다. (예를 들어, latent의 첫 번째 element를 바꾸면 성별이 바뀌는 것과 같이)
    - z는 prior distribution에서 얻어내고($ z \sim p(z) $)
    - 세타로 파라미터된 Deterministic한 네트워크(Decoder)에 들어갈 것이다. ($g_{\theta}(\cdot)$)
    - 그리고 우리는 p(x)를 알고 싶기 때문에 네트워크에서 추정하는 $p(x|g_{\theta}(z))$ 와 prior $ p(z) $ 를 활용해 $ \int p(x|g_{\theta}(z))p(z) dz = p(x) $ 을 구할 것이다.
    - z를 control 할 것이기 때문에 $p(z)$를 다루기 쉬운 normal이나 uniform 분포를 선택해 사용한다.
    
- 그러면 AE관점에서 latent가 manifold인 부분일 텐데, 이렇게 다루기 쉬운 분포를 사용해도 되냐.
    - https://arxiv.org/pdf/1606.05908 의 Figure2를 보면, 
    - 우리의 Decoder는 DNN이기 때문에 Decoder의 앞의 한 두개 Layer 정도가 Encoder의 역할을 수행할 수 있으니까 걱정하지 말라 한다.

- MLE로 directly하게 사용하면 안되는 것인지?
    - $ p(x|g_{\theta}(z))p(z) $ 이 두 값을 아니까, 샘플을 여러 개 모아서, summation을 통해 probability density estimation 할 수 있지 않은가. $ p(x) \approx \sum_{i} p(x|g_{\theta}(z))p(z) $ ?
    - 이게 안되니까 VAE를 사용해야한다. (- https://arxiv.org/pdf/1606.05908 의 Figure3) 
    - 우리의 train DB에 있는 x에 대하여, p(x)의 값이 높게 되기를 바라는 것에서
    - <img  src="./img/AAA-12.PNG" width="40%">
    - b(살짝 깎은 것)와 a는 의미적으로 a와 c(shifting한 것) 더 멀어야 하지만, a와 c의 Loss가 더 크다. MSE에서 
    - 다른 말로는 sampling 함수를 잘 조절줘야한다, 그냥 prior 분포에서 sampling을 하면은 제대로 학습이 안된다는 것이다. 


- 그래서 VAE에서 그냥 prior에서 샘플링하는 것이 아니라 x와 유의미하게 유사한 샘플이 나올 수 있는 확률분포  $ p(z|x) $에서 샘플링을 하려고 하지만 $ p(z|x) $를 알 수 없으니 Variational Inference를 활용한다.
    - VI는 우리가 모르는 True Posterior를 추정할 때 우리가 다루기 쉬운 함수 분포(Approximation Class)의 파라미터($\phi$) (ex. 가우시안의 평균, 분산)를 바꾸어가며 True Posterior를 추정한다.
    - $ p(z|x) \approx q_{\phi}(z|x) $
    - 그래서 Generator (decoder)를 학습할 때 학습을 잘 되게 하기 위한 z 샘플을 잘 만들어내는 이상적인 sample함수를 활용한다.
    - 그래서 지금까지 나온 것을 종합해보면 $p(x)$, $p(z|x)$, $q_{\phi}(z|x)$의 관계를 구하는 방식이 크게 2가지가 있다. 2번 째 방법이 제일 자료가 많이나오고 의미적으로 직관적이다. 그리고 이 관계식이 **Evidenc LowerBOund(ELBO)** 라는 것을 찾고자 하는 것이다.
    
    
- a. Jensen's Inequality를 활용해 ELBO를 구한다.
    - <img  src="./img/AAA-13.PNG" width="90%">
    - 직관적이진 않다.
    
- b. <img  src="./img/AAA-14.PNG" width="90%">
    - 옌센 부등식 안쓰면 KL과 ELBO가 나오는데 ELBO는 아까 구한 ELBO이다.
    - KL은 항상 양수이니까가고 p(x)를 maximize하려면 VI관점에서 KL을 minimize하면 되고 또 이것은 ELBO를 maximize하면 되는 것이다.
        - $ q_{\phi^{*}}(z|x) = argmax_{\phi} ELBO(\phi) $
    - 그리고 ELBO를 다시 전개해보면 또 KL이 나온다.
    - <img  src="./img/AAA-15.PNG" width="70%">


- 종합해보면, 우리는 2개의 최적화 문제 한꺼번에 푸는 것이다.
    - Optimization Problem 1 on $ \phi $: Variational Inference (ELBO를 maximize하는 $\phi$
        - $log(p(x) \ge \mathbb{E}_{q_{\phi}(z|x)} {\left[ log(p(x|z)) \right]} - {KL(q_{\phi}(z|x) \parallel p(z) )} = ELBO(\phi) $ 
    - Optimization Problem 2 on $\theta$: Maximum Likelihood (CH1에서 계속 얘기 했었던 것)
        - $ -\sum_{i} log(p(x_{i})) \le -\sum_{i} {\left\{ \mathbb{E}_{q_{\phi}(z|x_{i})} {\left[ log(p(x_{i}|{\color{Blue}g_{\theta}(z)})) \right]} - {KL(q_{\phi}(z|x_{i}) \parallel p(z) )} \right\}} $
    - Final Optimization Problem
        - $ \to argmin_{\phi,\theta} \sum_{i} - {\mathbb{E}_{q_{\phi}(z|x_{i})} {\left[ log(p(x_{i}|{g_{\theta}(z)})) \right]} + {KL(q_{\phi}(z|x_{i}) \parallel p(z) )} } $
        - 그리고 합친 이것 역시 ELBO를 Maximize하는 것과 같아진다.
        - ELBO식에 이미 포함되어 있기 때문에 (위 그림의 마지막 줄)
        - 그래서 세타를 포함하여 maximize 하면 위의 Final Optimization 식이 나온다.
    - <img  src="./img/AAA-16.PNG" width="70%">
    - 그래서 Decoder에 Encoder가 등장하게 되며 수학적으로는 관계가 없다는 것이다.


### Loss Function

- 다시 Loss를 자세히 보면
    - <img  src="./img/AAA-17.PNG" width="80%">

- a. Regularization (ELBO term)을 자세히 보자. $q_{\phi}(z|x_{i}), p(z)$는 다루기 쉬운 다음의 분포를 따른다고 정해준다.
    - 가정 1: Encoder(approximation class): multivariate gaussian distribution with a diagonal covariance
        - $q_{\phi}(z|x_{i}) \sim N(\mu_{i}, \sigma_{i}^{2}I)$
    - 가정 2: prior: multivariate normal distribution
        - $p(z) \sim N(0,I)$
    - $\to$ 이렇게 가정해주면, 가우시안 2개에 대한 KL은 다음과 같이 계산한다고 이미 정리된 것을 사용할 수 있다.
    - <img  src="./img/AAA-18.PNG" width="90%">
    - 코드에도 <pre><code>KL_divergence = 0.5 * tf.reduce_sum(tf.square(mu) + tf.square(sigma) - tf.log(1e-8 + tf.square(sigma)) - 1, 1) </code></pre>로 수식 그대로 들어가 있다. 어차피 최적화 문제들이니 간혹 상수들이 없는 코드 들도 있다.

- b. Reconstruction Error
    - ${\mathbb{E}_{q_{\phi}(z|x_{i})}{\left[ log(p_{\theta}(x_{i}|z)) \right]}}$을 구할 때 기댓값은 적분 $ \int log(p_{\theta}(x_{i}|z))q_{\phi}(z|x_{i}) dz  $으로 구해야하지만 
    - Monte-carlo technique으로 L개를 샘플링하여 계산한다. $ \to {{1}\over{L}}{\sum_{z^{i,l}} log\left(p_{\theta}( x_{i}|z^{i,l})\right) } $
        - 즉 정해진 $q_{\phi}$에서 $\mu_{i}$와 $\sigma_{i}$를 정했으므로 여기서 sampling을 L개(근데 주로 L개 샘플링하지 않고 그냥 하나씩 샘플링 한다.) 샘플링을 하여, Reconstruction Error를 정할 것이라는 것이다.
    - 하지만 샘플링한다는 것은 random이기 때문에 chain rule을 사용해야하는 backpropa를 사용하지 못하지만
        - Reparameterization Trick을 활용해 이 문제를 해결한다.
        - <img  src="./img/AAA-19.PNG" width="90%">
    - <pre><code>z = mu + sigma * tf.random_normal(tf.shape(mu), 0, 1, dtype=tf.float32)
    # z = tf.random_normal(tf.shape(mu), mu, sigma, dtype=tf.float32)</code></pre>
        - 그래서 코드에 다음과 같이 들어가는데 Reparameterization 안해도 잘 되긴한다. 왜냐하면 TF의 api를 까보면 내부적으로Reparameterization Trick을 활용한다.
    - 최종적으로 L을 1로 정하고 log-likelihood인 조건부 확률을 CH1에서 말했듯이 베르누이로하면 CE, 가우시안으로하면 MSE가 된다.
    - <img  src="./img/AAA-20.PNG" width="100%">


- 다시 총 정리하면 다음과 같다. 코드와 비교해 보자. [코드](https://github.com/hwalsuklee/tensorflow-mnist-VAE/blob/master/vae.py)
    - <img  src="./img/AAA-21.PNG" width="100%">


### Result:
- z의 dim을 늘릴 수록 성능이 좋아지고, 
- zero-masking noise with 50% prob.(꺼주고) + salt&peppr noise with 50% prob.(끄거나 더해주는)의 노이즈를 추가해도 복원을 탁월한 denoising 효과를 보인다.


### AE vs. VAE

- 코드를 보면 AE와 VAE는 거의 **KL term**만 다르다.(샘플링하는 것도 다르긴하지만) 그리고 그 KL term의 역할은 아래 그림과 같다.
    - <img  src="./img/AAA-22.PNG" width="100%">
    - 계속 말했듯이 AE는 데이터 압축이 목적이였고, VAE는 생성이 목적이었다. 
    - AE는 Encodeer만 사용하면 되는데, VAE는 이상적인 샘플링함수가 prior랑 같다고 하고, 그 prior를 잘 해석하는 encoder가 같이 학습되었기 때문에 그냥 normal dist로 위처럼 나온다.
        - 그래서 Generation할 때, AE는 z값을 어떻게 샘플링 해야하는지는 모르지만 VAE는 prior에서 샘플링을 하면 된다. (이것이 generation에서의 큰 차이)

- Learned Manifold 
    - prior에서 샘플링을 해보면, 저절로 굵기, 기울기와 같은 feature를 학습한 것을 볼 수 있다.
    - <img  src="./img/AAA-23.PNG" width="100%">


### Conditional VAE

- label을 알고 있으니 사용하자.
    - <img  src="./img/AAA-24.PNG" width="100%">
    - $log(p_{\theta}(x,y)) = log \int p_{\theta}(x,y|z) {{p(z)}\over{q_{\phi(z|x,y)}}} q_{\phi(z|x,y)}dz$
    - $ \qquad \qquad \quad \ge \int log \left( p_{\theta}(x,y|z) {{p(z)}\over{q_{\phi(z|x,y)}}} \right) q_{\phi(z|x,y)}dz $
    - $ \qquad \qquad \quad = \int log \left( p_{\theta}(x |y,z) {{p(y)p(z)}\over{q_{\phi(z|x,y)}}} \right) q_{\phi(z|x,y)}dz $
    - $ \qquad \qquad \quad = \mathbb{E}_{q_{\phi}(z|x,y)} \left[ log(p_{\theta}(x |y,z)) + log(p(y)) + log(p(z)) - log(q_{\phi}(z|x,y)) \right] $
    - $ \qquad \qquad \quad = -\mathcal{L}(x,y) $
    
- semi-supervised version은 아래와 같이 y를 추정하는 network를 별도로 두고, 아는 레이블은 그대로 y를 사용하면 되고 모르는 레이블은 추정해서 사용한다. (M2)
    - <img  src="./img/AAA-25.PNG" width="100%">
    - M3구조는 VAE 구조로 먼저 학습하고, 윗단에 layer하나 붙여서 y를 추정하라 했더니 잘 추정했다는 내용이다. 
    
- 코드는 Encoder와 Decoder의 input에 concat해주는 것만 다르다. (윗 윗 그림의 M2 일 때)
    - <pre><code>
    # gaussian_MLP_conditional_encoder
    input = tf.concat(axis=1, values=[x, y])
    # bernoulli_MLP_conditional_decoder
    input = tf.concat(axis=1, values=[z, y])
    </code></pre>


- Result
    - CVAE가 VAE를 비교하면 Denoise와 generate할 때, label 정보 때문에 더 빨리 수렴한다.
    - 또한, class를 fix하기 때문에 나머지 2개 (기울기, 굵기)를 알아서 학습하는 결과를 볼 수 있다. 숫자 글씨 style도 고정할 수 있다.
    - 그리고 CVAE의 manifold를 보면 굉장히 Entangled됫다고 볼 수 있지만, 레이블을 알려주었기 때문에 사실 이것은 정규분포로 잘 학습 된 것이고 함께 보여주려고 하기 때문인 것이다.
    - 그리고 semi-supervised를 할 때 50,000 개의 데이터에서 100개만 label을 주고 분류 성능을 보면 95%가 나온다. 
    

### Adversarial AE

- <img  src="./img/AAA-26.PNG" width="70%">
- VAE의 다양한 구조체들을 보면 Loss의 KL term이 가우시안이어야 계산하기 쉬워, 다 Decoder를 변형하는 것이었는데 이 한계를 극복하기 위한 것이 AAE이다.
    - 그래서 샘플링은 가능하지만 KL은ㄴ 계산해도 되지 않는 것을 써보려고 한다.
    - KL을 최적호 한다는 것은 $q_{\phi}()$가 $p(z)$와 같게 맞춰주는 것인데 이것이 GAN이 하는 것과 같다.
    - 그래서 KL 대신에 GAN Loss를 쓰면 임의의 함수가 가능하는 것으로 접근.

- Generative Adversarial Networks
    - <img  src="./img/AAA-27.PNG" width="70%">
    - VAE는 분포를 정하는 Expllicit Density에 속하고, GAN은 Implicit Density에 속하는 분포를 정하지 않지만 찾을 수 있는 방법론이다.
    - <img  src="./img/AAA-28.PNG" width="80%">
    - VAE는 ELBO를 maximize해 협력해서 학습하지만 GAN은 G, D의 힘의 평형을 맞추기 힘들다.
    - <img  src="./img/AAA-29.PNG" width="80%">
    - '+'는 진짜 '-' 는 가짜

- AAE의 Loss: 그래서 KL term 대신에 GAN Loss를 추가해 학습한다.
    - <img  src="./img/AAA-30.PNG" width="80%">
    - 그리고 학습할 때 G, D 번갈아 가면서 학습한다. 근데 Genration이 판별하는 것보다 더 어렵기 때문에 D한 번 학습할 때 G를 2번 학습하기도 한다. (판별은 찍어도 1/2이니까)
    

- VAE vs. AAE: (114 slide)
    - 두 모델의 prior $p(z)$를 시각화 해보면: VAE는 자주 나오는 값을 파악하는 것 중시하여 빈 공간이 가끔 있는 반면(Maximum liklihood의 관점에서 학습하여 샘플의 분포를 중시), AAE는 분포의 모양을 중시하여 빈 공간이 상대적으로 적다.(GAN lossd의 영향) 
    - p(z)를 10개의 mixuture of 10 gaussians라고 하고 10개의 날개에 label 정보를 discriminator에 사용하면 날개마다 특정 label의 이미지는 Latent space에서 의도된 구간으로 맵핑된다. 
        - 그래서 10개의 별모양, 스위스롤 모양으로 만들어 줄 수 있다.
    - **즉, manifold를 우리가 원하는 모양으로 만들 수 있다는 것이 장점.**
    - 논문에서 AAE의 Generator는 VAE보다 AE가 더 좋았다고 papar에서 말했다. 


- 추가로 샘플링 또한 안하고 할 수 있는 논문도 있다. Auto-Encoding Variational Bayes?
    


## Ch 5. Applications

- Text
    - [Semantic Hashing](http://www.cs.utoronto.ca/~rsalakhu/papers/semantic_final.pdf) 
    - [Dynamic Auto-Encoders for Semantic Indexing](http://yann.lecun.com/exdb/publis/pdf/mirowski-nipsdl-10.pdf) 

- Image
    - [Using Very Deep Autoencoders for Content-Based Image Retrieval](http://nuyoo.utm.mx/~jjf/rna/A6%20Using%20Very%20Deep%20Autoencoders%20for%20ContentBased%20Image%20Retrieval.pdf) 
    - [Autoencoding the Retrieval Relevance of Medical Images](https://arxiv.org/pdf/1507.01251.pdf) 

- Sound
    - [Retrieving Sounds by Vocal Imitation Recognition](http://www.ece.rochester.edu/~zduan/resource/ZhangDuan_RetrievingSoundsByVocalImitationRecognition_MLSP15.pdf)

- 3D model
    - [Deep Learning Representation using Autoencoder for 3D Shape Retrieval](https://arxiv.org/pdf/1409.7164.pdf) 
    - [Deep Signatures for Indexing and Retrieval in Large Motion Databases](http://web.cs.ucdavis.edu/~neff/papers/MIG_2015_DeepSignature.pdf) 
    - [DeepShape: Deep Learned Shape Descriptor for 3D Shape Matching and Retrieval](http://www.cvfoundation.org/openaccess/content_cvpr_2015/papers/Xie_DeepShape_Deep_Learned_2015_CVPR_paper.pdf) 

- Multi-modal
    - [Cross-modal Retrieval with Correspondence Autoencoder](https://people.cs.clemson.edu/~jzwang/1501863/mm2014/p7-feng.pdf) 
    - [Effective multi-modal retrieval based on stacked autoencoders](http://www.comp.nus.edu.sg/~ooibc/crossmodalvldb14.pdf) 




- 차원축소 기능으로 비교하면 AE나 VAE나 진배 없고, 생성이나 Denoising은 VAE가 KL term 때문에 더 좋다. 

- 차원축소 기능 때문에 Retrieval에서 많이 쓰인다. (네이버 검색에서도 사용)
    - AE들은 Multi-modal한 도메인에서도 한 곳으로 맞출 정도로 진짜 차원축소를 잘한다.(GAN은 없다.) 
    

   

- ex) Gray Face Generation
    - http://vdumoulin.github.io/morphing_faces/online_demo.html
    - GAN과 다르게 블러리하다. (pixe-wise 때문이라는 해석이 많다.)
    - 그래서 Reconstruction Loss를 CNN을 태워서 생성하는 논문 Deep Feature Consistent Variational Autoencoder

- ex) VAE의 Encode Decoder에 RNN을 붙인 Sketch RNN, AI가 Quickdraw를 맞춘다.
    - https://magenta.tensorflow.org/sketch-rnn-demo


### GAN vs. VAE
     
- <img  src="./img/AAA-31.PNG" width="100%">

- <img  src="./img/AAA-32.PNG" width="80%">
- 실선 3개가 실제 이미지 공간에서의 샘플들이고, 이미지 하나가 2d공간의 한 점
- VAE는 Expllicit Density를 추정하니 잘 아우르게 학습이되지만 blurry하다.
- GAN은 Generator가 만들어 내는 애를 Discriminator가 진짜 데이터와 구분 못하면 끝나니까 진짜 같게 잘 만들지만 Mode collapse가 일어날 가능성이 있다.


- 이활석님 GAN 코드, Loos 정리 링크
    - https://github.com/hwalsuklee/tensorflow-generative-model-collections
    - 크게 학습이 잘되게하기 위한 논문 그리고 Application에 적용한 논문 2개가 있다.
    

- 또 VAE와 GAN을 같이 사용하는 것이 추세다. EBGAN, BEGAN
    - Discriminator를 오토인코더로 한 것.
    
    
- Stack GAN: Multimodal Featur Learner (3/3 21분)
    - 글을 주면 알맞은 이미지 생성
    - <img  src="./img/AAA-33.PNG" width="100%">
    
    
- 이외에 3D모델로 생성, 음성 노이즈 제거, Age Progression, 채색 변환, 16x16의 이미지를 128x128로 복원, Fashion Generate 등이 있다.


- 지금 GAN은 1024 x 1024 이미지까지 잘 generate 한다.



## Reference

- Lecture:
    - https://tv.naver.com/v/3185672?query=%EC%98%A4%ED%86%A0%EC%9D%B8%EC%BD%94%EB%8D%94%EC%9D%98+%EB%AA%A8%EB%93%A0%EA%B2%83&plClips=false:3185672:3185730:3186249
- Slide:
    - https://www.slideshare.net/NaverEngineering/ss-96581209
- Code:
    - https://github.com/hwalsuklee/tensorflow-mnist-VAE/blob/master/vae.py
    - https://github.com/hwalsuklee/tensorflow-mnist-CVAE/blob/master/vae.py
    - https://github.com/hwalsuklee/tensorflow-mnist-AAE